In [ ]:
%matplotlib inline

# Multi-start optimization.

The optimization algorithm `multistart`
generates starting points using a DOE algorithm
and run a sub-optimization algorithm from each starting point.


In [ ]:
from __future__ import annotations

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import create_scenario
from gemseo import execute_post
from gemseo.algos.opt.multi_start.settings.multi_start_settings import (
    MultiStart_Settings,
)

First,
we create the disciplines



In [ ]:
objective = create_discipline("AnalyticDiscipline", expressions={"obj": "x**3-x+1"})
constraint = create_discipline(
    "AnalyticDiscipline", expressions={"cstr": "x**2+obj**2-1.5"}
)

and the design space



In [ ]:
design_space = create_design_space()
design_space.add_variable("x", lower_bound=-1.5, upper_bound=1.5, value=1.5)

Then,
we define the MDO scenario



In [ ]:
scenario = create_scenario(
    [objective, constraint],
    "obj",
    design_space,
    formulation_name="DisciplinaryOpt",
)

Note that the formulation settings passed to [create_scenario()][gemseo.create_scenario] can be provided
via a Pydantic model. For more information, see [this page][formulation-settings].



In [ ]:
scenario.add_constraint("cstr", constraint_type="ineq")

and execute it with the `MultiStart` optimization algorithm
combining the local optimization algorithm SLSQP
and the full-factorial DOE algorithm:



In [ ]:
multistart_settings = MultiStart_Settings(
    max_iter=100,
    opt_algo_name="SLSQP",
    doe_algo_name="PYDOE_FULLFACT",
    n_start=10,
    # Set multistart_file_path to save the history of the local optima.
    multistart_file_path="multistart.hdf5",
)
scenario.execute(multistart_settings)

Lastly,
we can plot the history of the objective,
either by concatenating the 10 sub-optimization histories:



In [ ]:
execute_post(
    scenario, post_name="BasicHistory", variable_names=["obj"], save=False, show=True
)

or by filtering the local optima (one per starting point):



In [ ]:
execute_post(
    "multistart.hdf5",
    post_name="BasicHistory",
    variable_names=["obj"],
    save=False,
    show=True,
)